In [1]:
import pandas as pd
import os

In [2]:
ruta_archivos = [os.path.join('historicos_cubo', ruta) for ruta in os.listdir('historicos_cubo')]
ruta_archivos = sorted(ruta_archivos, key=lambda x: int(x.split('_')[3]))

In [3]:
dfs_cubo = {'_'.join(ruta.split('_')[4: 6]): pd.read_excel(ruta) for ruta in ruta_archivos}

In [21]:
cc_a_obtener = ['66-HOSPITALIZACIÓN MEDICINA INTERNA', '90-HOSPITALIZACIÓN QUIRÚRGICA',
                '102-HOSPITALIZACIÓN CARDIOVASCULAR', '464-QUIRÓFANOS CARDIOVASCULAR',
                '484-QUIRÓFANOS TORACICA', '166-UNIDAD DE CUIDADOS INTENSIVOS',
                '195-UNIDAD DE TRATAMIENTO INTENSIVO ADULTO']

historico_cubo = {}
for fecha_df, df in dfs_cubo.items():
    df = df.set_index('Insumos / Centro Costos')
    gasto_total = df.loc['Total Directos', cc_a_obtener].to_frame()
    gasto_total['Total Indirectos'] = df.loc['Total Indirectos', cc_a_obtener]
    gasto_total['Fecha'] = pd.to_datetime(fecha_df, format='%Y_%m')


In [22]:
gasto_total

,Total Directos,Total Indirectos,Fecha
66-HOSPITALIZACIÓN MEDICINA INTERNA,232608519,313680558,2022-09-01
90-HOSPITALIZACIÓN QUIRÚRGICA,52684522,42134361,2022-09-01
102-HOSPITALIZACIÓN CARDIOVASCULAR,124895141,641785094,2022-09-01
464-QUIRÓFANOS CARDIOVASCULAR,269689731,470206538,2022-09-01
484-QUIRÓFANOS TORACICA,147708156,97426357,2022-09-01
166-UNIDAD DE CUIDADOS INTENSIVOS,162409596,62124871,2022-09-01
195-UNIDAD DE TRATAMIENTO INTENSIVO ADULTO,295502124,83697830,2022-09-01


In [11]:
gastos_tavi = pd.DataFrame.from_dict(historico_cubo).transpose().reset_index()
gastos_tavi.columns = ['Fecha', 'Gasto_Indirecto']
gastos_tavi['Fecha'] = pd.to_datetime(gastos_tavi['Fecha'], format='%Y_%m')

In [12]:
gastos_tavi

,Fecha,Gasto_Indirecto
0,2021-01-01,...
1,2021-02-01,...
2,2021-03-01,...
3,2021-04-01,...
4,2021-05-01,...
5,2021-06-01,...
6,2021-07-01,...
7,2021-08-01,...
8,2021-09-01,...
9,2021-10-01,...


In [15]:
ruta_archivos = [os.path.join('historicos_producciones', ruta) for ruta in os.listdir('historicos_producciones')]
ruta_archivos = sorted(ruta_archivos)


In [16]:
dfs_produccion = {ruta.split()[-2]: pd.read_excel(ruta) for ruta in ruta_archivos}

In [22]:
columnas = {"Unnamed: 2": "ENERO",
            "Unnamed: 3": "FEBRERO",
            "Unnamed: 4": "MARZO",
            "Unnamed: 5": "ABRIL",
            "Unnamed: 6": "MAYO",
            "Unnamed: 7": "JUNIO",
            "Unnamed: 8": "JULIO",
            "Unnamed: 9": "AGOSTO",
            "Unnamed: 10": "SEPTIEMBRE",
            "Unnamed: 11": "OCTUBRE",
            "Unnamed: 12": "NOVIEMBRE",
            "Unnamed: 13": "DICIEMBRE"}

In [26]:
dfs_produccion['2021'] = dfs_produccion['2021'].rename(columns=columnas)
dfs_produccion['2022'] = dfs_produccion['2022'].rename(columns=columnas)

In [64]:
tavi_2021 = dfs_produccion['2021'].loc[62]['ENERO':'DICIEMBRE']
tavi_2022 = dfs_produccion['2022'].loc[62]['ENERO':'DICIEMBRE']

cantidad_tavis = pd.concat([tavi_2021, tavi_2022]).reset_index().drop(columns=['index'])

In [70]:
gastos_tavi['Cantidad_TAVIs'] = cantidad_tavis.astype('Int32')
gastos_tavi = gastos_tavi.replace(0, 0)

In [71]:
gastos_tavi['Gasto_Indirecto_promedio'] = gastos_tavi['Gasto_Indirecto'].divide(gastos_tavi['Cantidad_TAVIs'])

In [73]:
gastos_tavi.to_excel('tavi_indirectos_historicos.xlsx', index=False)